In [1]:
import pandas as pd
import torch
import yaml
from torch import cosine_similarity

from model.encoder import Encoder
from model.recommender import DeepFM
from utils.misc import send_to_device

This can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64

2024-10-26 05:03:35.531053: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-26 05:03:35.535656: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-26 05:03:35.543007: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-26 05:03:35.545

In [2]:
movies = pd.read_csv("data/ml-20m/movies.csv")

movies = movies[["item_id", "item_title"]]

num_items = movies["item_id"].nunique()

requests = pd.read_csv('data/ml-20m/requests.csv')
requests = requests.groupby("item_id").agg({
    "item_title": "first",
    "request": list,
}).reset_index()
requests.set_index("item_id", inplace=True, drop=False)

In [ ]:
args = yaml.safe_load(open("configs/collaborative.yaml", "r"))
args["recommender"]["weights"] = "weights/collaborative/deepfm.pt"
args["encoder"]["weights"] = "weights/collaborative/encoder.pt"

device = "cuda" if torch.cuda.is_available() else "cpu"

encoder = Encoder(**args["encoder"]).to(device)

recommender = DeepFM(num_items=num_items, **args["recommender"]).to(device)

item_embeddings = recommender.embedding.item_embedding.weight.cpu()

In [ ]:
features = (torch.tensor([num_items, 0]), torch.tensor([5.0, 3.5]))

features = send_to_device(features, device)

user_ratings = recommender.predict(features).cpu()

# Join user_ratings on movies
ranked_movies = movies.join(pd.DataFrame(user_ratings * 5, columns=["rating"]))

request = "I want to watch a popular romantic comedy from the 80s"

request_embedding = encoder(request).cpu()

similarities = (cosine_similarity(request_embedding, item_embeddings) + 1) / 2

rankings = user_ratings * similarities

_, indices = torch.topk(rankings, k=10, largest=True)

ranked_movies.iloc[indices]